In [29]:
import autogen
# ollama run deepseek-r1:1.5b
config_list = [
    {
        "model": "deepseek-r1:1.5b",
        "base_url": "http://localhost:11434/v1",
        "api_key": "ollama",
    }
]

user_input_agent = autogen.UserProxyAgent(
    name="User",
    system_message="Provide a brief description.",
    code_execution_config={"last_n_messages" : 2, "work_dir": "groupchat", "use_docker": False},
    human_input_mode= "NEVER",
    is_termination_msg=lambda x: x.get("content", "").rstrip().endswith("TERMINATE")
)

planning_agent = autogen.AssistantAgent(
    name="Planner",
    system_message="Based on the given situation, identify the gaps",
    llm_config={"config_list": config_list},
)

recommendation_agent = autogen.AssistantAgent(
    name="Recommender",
    system_message="Based on the Planner findings, provide a set of process improvement recommendations and end the conversation",
    llm_config={"config_list": config_list},
)

def state_transition(last_speaker, groupchat):
    messages = groupchat.messages

    if last_speaker is user_input_agent:
        return planning_agent

    elif last_speaker is planning_agent:
        return recommendation_agent

    elif last_speaker is recommendation_agent:
        return None


# Initialize GroupChat
groupchat = autogen.GroupChat(
    agents=[
        user_input_agent, planning_agent, recommendation_agent
    ],
    max_round = 4,
    speaker_selection_method = state_transition,
    messages=[]
)

manager = autogen.GroupChatManager(groupchat=groupchat)

user_input_agent.initiate_chat(manager, message = "We are experiencing delays in final assembly, and our production output has dropped by 15 percent in the last three months. I suspect bottlenecks in component delivery, machine downtime, or inefficient worker allocation. Can you help analyze the situation and suggest improvements?")

User (to chat_manager):

We are experiencing delays in final assembly, and our production output has dropped by 15 percent in the last three months. I suspect bottlenecks in component delivery, machine downtime, or inefficient worker allocation. Can you help analyze the situation and suggest improvements?

--------------------------------------------------------------------------------

Next speaker: Planner

[autogen.oai.client: 05-03 12:18:21] {573} WARNING - Model deepseek-r1:1.5b is not found. The cost will be 0. In your config_list, add field {"price" : [prompt_price_per_1k, completion_token_price_per_1k]} for customized pricing.
Planner (to chat_manager):

<think>
Alright, I need to figure out how to help this user with their situation. Let me start by understanding what they're dealing with.

They mentioned that they've experienced delays in final assembly and an 15% drop in production output over the last three months. That's a significant issue on its own. The possible causes 

ChatResult(chat_id=None, chat_history=[{'content': 'We are experiencing delays in final assembly, and our production output has dropped by 15 percent in the last three months. I suspect bottlenecks in component delivery, machine downtime, or inefficient worker allocation. Can you help analyze the situation and suggest improvements?', 'role': 'assistant', 'name': 'User'}, {'content': "<think>\nAlright, I need to figure out how to help this user with their situation. Let me start by understanding what they're dealing with.\n\nThey mentioned that they've experienced delays in final assembly and an 15% drop in production output over the last three months. That's a significant issue on its own. The possible causes are bottlenecks in component delivery, machine downtime, or inefficient worker allocation. My task is to identify gaps here and suggest improvements.\n\nFirst, I should break down each cause of the bottleneck. For comp competition, maybe the supply chain was an issue where compone

In [ ]:
# We are experiencing delays in final assembly, and our production output has dropped by 15 percent in the last three months. I suspect bottlenecks in component delivery, machine downtime, or inefficient worker allocation. Can you help analyze the situation and suggest improvements?